In [34]:
from functions_for_all_nutrients import *
from vitamin_a_and_zinc_functions import *

In [35]:
# Get absolute path of repo based on relative directory structure, so it should work for all users
import os.path
vivarium_research_lsff_path = os.path.abspath("..")
vivarium_research_lsff_path

'/ihme/homes/alibow/notebooks/vivarium_research_lsff'

In [36]:
username = !whoami
username

['alibow']

In [37]:
# Specify directory with saved GBD data so that it's not necessary to call get_draws on every run.

# Nathaniel has saved some .hdf files in this directory on /share
# You can specify a different directory if you have GBD data saved somewhere else.
# Or if you want to re-pull the data, you can change this to a nonexistent directory.
gbd_data_directory = '/share/scratch/users/ndbs/vivarium_lsff/gbd_data'
gbd_data_filepath = f'{gbd_data_directory}/multmodel_data.hdf'

In [38]:
index_cols=['location_id','sex_id','age_group_id']
age_group_ids = [2,3,4,5]
sex_ids = [1,2]
coverage_levels = [0.2,0.5,0.8]
years = [2021,2022,2023,2024,2025]

In [39]:
coverage_data_dir = vivarium_research_lsff_path + '/data_prep/outputs/lsff_input_coverage_data.csv'
# # Old version that may not work for all users:
# coverage_data_dir = f'/ihme/homes/{username[0]}/notebooks/' \
#     'vivarium_research_lsff/data_prep/outputs/lsff_input_coverage_data.csv'

In [40]:
location_ids = (list(
                pd.read_csv(coverage_data_dir)
                .location_id
                .unique()))
location_ids

[179,
 163,
 214,
 161,
 165,
 190,
 189,
 196,
 201,
 180,
 15,
 164,
 20,
 202,
 205,
 171,
 11,
 184,
 168,
 6,
 141,
 207,
 181,
 213,
 522]

# VITAMIN A

In [41]:
# vitamin A specific -- these should be replaced for other models
rei_id = 96
nutrient = 'vitamin a'
vehicles = ['oil','maize flour','wheat flour']
effective_fractions = [0, 0, (365 - (365 * 0.5)) / (365 - 28), 1]

In [42]:
# define no fortification relative risk distribution
# vitamin a specific -- this should be replaced for other models

from numpy import log
from scipy.stats import norm, lognorm

# median and 0.975-quantile of lognormal distribution for RR
median = 2.22
q_975 = 5.26

# 0.975-quantile of standard normal distribution (=1.96, approximately)
q_975_stdnorm = norm().ppf(0.975)

mu = log(median) # mean of normal distribution for log(RR)
sigma = (log(q_975) - mu) / q_975_stdnorm # std dev of normal distribution for log(RR)

# Frozen lognormal distribution for RR, representing uncertainty in our effect size
# (s is the shape parameter)b
rr_distribution = lognorm(s=sigma, scale=median)

In [43]:
vitamin_a_baseline_coverage = pd.read_pickle('results_raw/baseline_coverage_data_u5.pkl')
vitamin_a_baseline_coverage = (vitamin_a_baseline_coverage.loc[vitamin_a_baseline_coverage.nutrient=='vitamin a']
                              .loc[vitamin_a_baseline_coverage.vehicle.isin(vehicles)]
                              .drop(columns='nutrient'))

vitamin_a_counterfactual_coverage = pd.read_pickle('results_raw/counterfactual_coverage_data_u5.pkl')
vitamin_a_counterfactual_coverage = (vitamin_a_counterfactual_coverage.loc[vitamin_a_counterfactual_coverage.nutrient=='vitamin a']
                              .loc[vitamin_a_counterfactual_coverage.vehicle.isin(vehicles)]
                                    .drop(columns='nutrient'))

vitamin_a_baseline_coverage.head()

,draw_0,draw_1,draw_10,draw_100,draw_101,draw_102,draw_103,draw_104,draw_105,draw_106,...,draw_993,draw_994,draw_995,draw_996,draw_997,draw_998,draw_999,location_id,year,vehicle
0,0.423812,0.430717,0.434723,0.449276,0.442398,0.416923,0.430786,0.444095,0.471659,0.441113,...,0.453842,0.443832,0.440586,0.447304,0.426667,0.430761,0.422643,11,2021.0,oil
1,0.423812,0.430717,0.434723,0.449276,0.442398,0.416923,0.430786,0.444095,0.471659,0.441113,...,0.453842,0.443832,0.440586,0.447304,0.426667,0.430761,0.422643,11,2022.0,oil
2,0.423812,0.430717,0.434723,0.449276,0.442398,0.416923,0.430786,0.444095,0.471659,0.441113,...,0.453842,0.443832,0.440586,0.447304,0.426667,0.430761,0.422643,11,2023.0,oil
3,0.423812,0.430717,0.434723,0.449276,0.442398,0.416923,0.430786,0.444095,0.471659,0.441113,...,0.453842,0.443832,0.440586,0.447304,0.426667,0.430761,0.422643,11,2024.0,oil
4,0.423812,0.430717,0.434723,0.449276,0.442398,0.416923,0.430786,0.444095,0.471659,0.441113,...,0.453842,0.443832,0.440586,0.447304,0.426667,0.430761,0.422643,11,2025.0,oil


In [44]:
vitamin_a_baseline_effective_coverage = get_effective_vitamin_a_coverage(vitamin_a_baseline_coverage.set_index(['location_id','year','vehicle']), 
                                                               sex_ids,
                                                               age_group_ids,
                                                               effective_fractions,
                                                               years)
vitamin_a_counterfactual_effective_coverage = get_effective_vitamin_a_coverage(vitamin_a_counterfactual_coverage.set_index(['location_id','year','vehicle','coverage_level']), 
                                                               sex_ids,
                                                               age_group_ids,
                                                               effective_fractions,
                                                               years)

In [45]:
vitamin_a_rr_deficiency_nofort = generate_rr_deficiency_nofort_draws(mu, sigma, location_ids)
vitamin_a_rr_deficiency_nofort.mean(axis=1)

location_id
179    2.396239
163    2.396239
214    2.396239
161    2.396239
165    2.396239
190    2.396239
189    2.396239
196    2.396239
201    2.396239
180    2.396239
15     2.396239
164    2.396239
20     2.396239
202    2.396239
205    2.396239
171    2.396239
11     2.396239
184    2.396239
168    2.396239
6      2.396239
141    2.396239
207    2.396239
181    2.396239
213    2.396239
522    2.396239
dtype: float64

In [46]:
vitamin_a_dalys_hdf_key = '/vitamin_a_deficiency/dalys_attributable_bmgf_25_countries'
try:
    vitamin_a_dalys = pd.read_hdf(gbd_data_filepath, key=vitamin_a_dalys_hdf_key)
except (FileNotFoundError, KeyError) as e:
    print(e, "Calling get_draws...")
    vitamin_a_dalys = pull_deficiency_attributable_dalys(rei_id,
                                                  location_ids,
                                                  age_group_ids,
                                                  sex_ids,
                                                  index_cols)
else:
    # Stored data included all 25 locations, so filter to desired locations if try clause succeeded
    vitamin_a_dalys = vitamin_a_dalys.query("location_id in @location_ids")

vitamin_a_dalys.head()

draw_0       draw_1      draw_10  \
location_id sex_id age_group_id                                          
6           1      2                6.518769    13.105468     3.094651   
                   3              137.319706   160.767128    94.890071   
                   4             2655.803809  1851.342551  1589.046172   
                   5             6014.654573  5196.950929  5416.861679   
            2      2                1.257434     0.399007     0.553913   

                                    draw_100     draw_101     draw_102  \
location_id sex_id age_group_id                                          
6           1      2               11.038021     4.401997    11.975454   
                   3               90.369198    60.432847    55.529898   
                   4             1599.004574  1346.298841  1256.386415   
                   5             5331.341355  3854.742854  3731.005037   
            2      2                1.023849     0.758328     2.072954   

                                    draw_103     draw_104     draw_105  \
location_id sex_id age_group_id                                          
6           1      2                7.494744     8.345612     6.911725   
                   3               90.291739   271.638485    98.426889   
                   4             2277.158107  1945.723118  1648.834016   
                   5             5998.981182  5148.786063  6713.467238   
            2      2                1.215976     0.498824     1.484751   

                                    draw_106  ...     draw_990     draw_991  \
location_id sex_id age_group_id               ...                             
6           1      2                3.246637  ...     7.744664     6.914776   
                   3              103.576291  ...    46.290755   114.677909   
                   4             1727.435654  ...  1401.487949  2418.174399   
                   5             4967.112706  ...  4836.679039  7762.494425   
            2      2                0.292424  ...     0.617292     2.122949   

                                    draw_992     draw_993     draw_994  \
location_id sex_id age_group_id                                          
6           1      2               16.271661     9.998869     8.958759   
                   3              162.199784    44.200716   132.494285   
                   4             1694.596125  1575.049825  1867.622230   
                   5             9917.792849  5999.923366  5959.441779   
            2      2                0.496779     0.652097     1.054172   

                                    draw_995     draw_996     draw_997  \
location_id sex_id age_group_id                                          
6           1      2               10.438845     4.813849     2.642819   
                   3               77.170521   111.370798    50.678223   
                   4             2990.603818   965.710245  1477.468702   
                   5             6283.476586  2741.025158  4291.649161   
            2      2                0.565635     0.495569     0.357408   

                                    draw_998     draw_999  
location_id sex_id age_group_id                            
6           1      2                6.619699     4.775778  
                   3              179.346982   140.162456  
                   4             1660.723099  1682.522484  
                   5             4393.900720  6320.589774  
            2      2                0.691123     0.653190  

[5 rows x 1000 columns]

In [47]:
vitamin_a_paf_deficiency_nofort = calculate_paf_deficiency_nofort(
                                    vitamin_a_rr_deficiency_nofort, 
                                    vitamin_a_baseline_effective_coverage)
vitamin_a_paf_deficiency_nofort.head()

draws                                                draw_0    draw_1  \
location_id sex_id age_group_id year   vehicle                          
6           1      2            2021.0 wheat flour  0.78595  0.447022   
                                2022.0 wheat flour  0.78595  0.447022   
                                2023.0 wheat flour  0.78595  0.447022   
                                2024.0 wheat flour  0.78595  0.447022   
                                2025.0 wheat flour  0.78595  0.447022   

draws                                                draw_10  draw_100  \
location_id sex_id age_group_id year   vehicle                           
6           1      2            2021.0 wheat flour  0.406811  0.340996   
                                2022.0 wheat flour  0.406811  0.340996   
                                2023.0 wheat flour  0.406811  0.340996   
                                2024.0 wheat flour  0.406811  0.340996   
                                2025.0 wheat flour  0.406811  0.340996   

draws                                               draw_101  draw_102  \
location_id sex_id age_group_id year   vehicle                           
6           1      2            2021.0 wheat flour  0.617978  0.620854   
                                2022.0 wheat flour  0.617978  0.620854   
                                2023.0 wheat flour  0.617978  0.620854   
                                2024.0 wheat flour  0.617978  0.620854   
                                2025.0 wheat flour  0.617978  0.620854   

draws                                               draw_103  draw_104  \
location_id sex_id age_group_id year   vehicle                           
6           1      2            2021.0 wheat flour  0.149863  0.636347   
                                2022.0 wheat flour  0.149863  0.636347   
                                2023.0 wheat flour  0.149863  0.636347   
                                2024.0 wheat flour  0.149863  0.636347   
                                2025.0 wheat flour  0.149863  0.636347   

draws                                               draw_105  draw_106  ...  \
location_id sex_id age_group_id year   vehicle                          ...   
6           1      2            2021.0 wheat flour  0.421241  0.759597  ...   
                                2022.0 wheat flour  0.421241  0.759597  ...   
                                2023.0 wheat flour  0.421241  0.759597  ...   
                                2024.0 wheat flour  0.421241  0.759597  ...   
                                2025.0 wheat flour  0.421241  0.759597  ...   

draws                                               draw_990  draw_991  \
location_id sex_id age_group_id year   vehicle                           
6           1      2            2021.0 wheat flour  0.507946  0.455066   
                                2022.0 wheat flour  0.507946  0.455066   
                                2023.0 wheat flour  0.507946  0.455066   
                                2024.0 wheat flour  0.507946  0.455066   
                                2025.0 wheat flour  0.507946  0.455066   

draws                                               draw_992  draw_993  \
location_id sex_id age_group_id year   vehicle                           
6           1      2            2021.0 wheat flour  0.740104  0.785748   
                                2022.0 wheat flour  0.740104  0.785748   
                                2023.0 wheat flour  0.740104  0.785748   
                                2024.0 wheat flour  0.740104  0.785748   
                                2025.0 wheat flour  0.740104  0.785748   

draws                                               draw_994  draw_995  \
location_id sex_id age_group_id year   vehicle                           
6           1      2            2021.0 wheat flour  0.396677  0.557149   
                                2022.0 wheat flour  0.396677  0.557149   
                                2023.0 whea

In [48]:
vitamin_a_pif_deficiency_nofort = calculate_pif_deficiency_nofort(
                                    vitamin_a_paf_deficiency_nofort,
                                    vitamin_a_baseline_effective_coverage,
                                    vitamin_a_counterfactual_effective_coverage)
vitamin_a_pif_deficiency_nofort.head()

draws                                                              draw_0  \
location_id sex_id age_group_id year   vehicle     coverage_level           
6           1      2            2021.0 wheat flour 0.2                0.0   
                                                   0.5                0.0   
                                                   0.8                0.0   
                                2022.0 wheat flour 0.2                0.0   
                                                   0.5                0.0   

draws                                                              draw_1  \
location_id sex_id age_group_id year   vehicle     coverage_level           
6           1      2            2021.0 wheat flour 0.2                0.0   
                                                   0.5                0.0   
                                                   0.8                0.0   
                                2022.0 wheat flour 0.2                0.0   
                                                   0.5                0.0   

draws                                                              draw_10  \
location_id sex_id age_group_id year   vehicle     coverage_level            
6           1      2            2021.0 wheat flour 0.2                 0.0   
                                                   0.5                 0.0   
                                                   0.8                 0.0   
                                2022.0 wheat flour 0.2                 0.0   
                                                   0.5                 0.0   

draws                                                              draw_100  \
location_id sex_id age_group_id year   vehicle     coverage_level             
6           1      2            2021.0 wheat flour 0.2                  0.0   
                                                   0.5                  0.0   
                                                   0.8                  0.0   
                                2022.0 wheat flour 0.2                  0.0   
                                                   0.5                  0.0   

draws                                                              draw_101  \
location_id sex_id age_group_id year   vehicle     coverage_level             
6           1      2            2021.0 wheat flour 0.2                  0.0   
                                                   0.5                  0.0   
                                                   0.8                  0.0   
                                2022.0 wheat flour 0.2                  0.0   
                                                   0.5                  0.0   

draws                                                              draw_102  \
location_id sex_id age_group_id year   vehicle     coverage_level             
6           1      2            2021.0 wheat flour 0.2                  0.0   
                                                   0.5                  0.0   
                                                   0.8                  0.0   
                                2022.0 wheat flour 0.2                  0.0   
                                                   0.5                  0.0   

draws                                                              draw_103  \
location_id sex_id age_group_id year   vehicle     coverage_level             
6           1      2            2021.0 wheat flour 0.2                  0.0   
                                                   0.5                  0.0   
                                                   0.8                  0.0   
                                2022.0 wheat flour 0.2                  0.0   
                                                   0.5                  0.0   

draws                                                              draw_104  \
location_id sex_id age_group_id year   vehicle     coverage_level             

In [49]:
vitamin_a_exposure = get_draws(gbd_round_id=6,
                              year_id=2019,
                              location_id=location_ids,
                              sex_id=sex_ids,
                              age_group_id=age_group_ids,
                              gbd_id_type='rei_id',
                              gbd_id=96,
                              source='exposure',
                              decomp_step='step4')

vitamin_a_exp = (vitamin_a_exposure.loc[vitamin_a_exposure.parameter=='cat1']
                 .set_index(['location_id','sex_id','age_group_id']))
vitamin_a_exp = vitamin_a_exp.drop(columns=[c for c in vitamin_a_exp.columns if 'draw' not in c])

vitamin_a_exp = vitamin_a_exp - (vitamin_a_exp * vitamin_a_pif_deficiency_nofort)

vitamin_a_exp = vitamin_a_exp.reset_index().merge(get_population(gbd_round_id=6,
                    location_id=location_ids,
                    sex_id=sex_ids,
                    age_group_id=age_group_ids,
                    year_id=2019,
                    decomp_step='step4'), on=['location_id','sex_id','age_group_id'])

for i in list(range(0,1000)):
    vitamin_a_exp[f'draw_{i}'] = vitamin_a_exp[f'draw_{i}'] * vitamin_a_exp.population
    
vitamin_a_exp = vitamin_a_exp.groupby(['location_id','vehicle','year','coverage_level']).sum()

for i in list(range(0,1000)):
    vitamin_a_exp[f'draw_{i}'] = vitamin_a_exp[f'draw_{i}'] / vitamin_a_exp.population * 100
    
vitamin_a_exp = (vitamin_a_exp.drop(columns=[c for c in vitamin_a_exp.columns if 'draw' not in c])
                 .reset_index())
vitamin_a_exp = vitamin_a_exp.loc[vitamin_a_exp.year==2025]#.drop(columns='year')
vitamin_a_exp['nutrient'] = 'vitamin A'

vitamin_a_exp

,location_id,vehicle,year,coverage_level,draw_0,draw_1,draw_10,draw_100,draw_101,draw_102,...,draw_991,draw_992,draw_993,draw_994,draw_995,draw_996,draw_997,draw_998,draw_999,nutrient
12,6,wheat flour,2025.0,0.2,2.389974,3.589469,3.364334,5.308845,3.007441,4.118048,...,3.937520,5.084646,3.411043,4.781297,4.619861,3.106205,1.868402,2.349117,5.329272,vitamin A
13,6,wheat flour,2025.0,0.5,2.199227,3.436788,3.099283,5.074053,2.700411,3.724679,...,3.608144,4.557853,3.014083,4.537453,4.316141,2.753669,1.708314,2.256979,4.967975,vitamin A
14,6,wheat flour,2025.0,0.8,2.008480,3.284107,2.834233,4.839260,2.393382,3.331309,...,3.278769,4.031060,2.617124,4.293610,4.012421,2.401133,1.548226,2.164841,4.606678,vitamin A
27,11,oil,2025.0,0.2,12.342328,13.387627,21.297428,16.143740,20.677608,13.649645,...,16.029681,15.926320,12.850385,15.381113,13.379445,14.769927,15.179865,13.820200,13.905487,vitamin A
28,11,oil,2025.0,0.5,10.486088,12.483173,19.900807,15.322969,18.320920,12.209519,...,14.733416,13.600500,10.739587,14.411514,12.084264,12.945496,14.122590,12.969363,13.087270,vitamin A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
943,522,oil,2025.0,0.5,15.228365,15.790132,16.809931,17.075943,24.194779,23.504951,...,16.608754,21.650482,22.103050,25.284084,20.117752,23.388306,18.600406,17.222088,20.801857,vitamin A
944,522,oil,2025.0,0.8,15.228365,15.790132,16.809931,17.075943,24.194779,23.504951,...,16.608754,21.650482,22.103050,25.284084,20.117752,23.388306,18.600406,17.222088,20.801857,vitamin A
957,522,wheat flour,2025.0,0.2,14.205117,14.875331,16.173427,16.397688,22.523829,22.142426,...,15.838585,19.058990,20.017024,23.804808,18.998923,21.583363,17.589538,16.632215,19.799175,vitamin A
958,522,wheat flour,2025.0,0.5,12.670246,13.503128,15.218673,15.380306,20.017405,20.098640,...,14.683331,15.171752,16.887984,21.585893,17.320679,18.875949,16.073236,15.747405,18.295153,vitamin A


In [50]:
vitamin_a_exp.to_pickle('results_raw/vitamin_a_deficiency_exposure.pkl')

In [51]:
vitamin_a_overall_pifs, vitamin_a_daly_reduction_counts = calculate_final_pifs_and_daly_reductions(
                                                    vitamin_a_pif_deficiency_nofort,
                                                    vitamin_a_dalys,
                                                    coverage_levels, years)

In [52]:
vitamin_a_daly_reduction_rates = calculate_rates(vitamin_a_daly_reduction_counts, 
                                            location_ids, 
                                            age_group_ids, 
                                            sex_ids)

In [53]:
vitamin_a_overall_pifs['measure'] = 'pif'
vitamin_a_daly_reduction_counts['measure'] = 'counts_averted'
vitamin_a_daly_reduction_rates['measure'] = 'rates_averted'
vitamin_a_results = pd.concat([vitamin_a_overall_pifs.reset_index(),
                         vitamin_a_daly_reduction_counts.reset_index(),
                         vitamin_a_daly_reduction_rates.reset_index()],
                        ignore_index=True)
vitamin_a_results.to_pickle('results_raw/vitamin_a.pkl')
vitamin_a_results

draws,location_id,year,vehicle,coverage_level,draw_0,draw_1,draw_10,draw_100,draw_101,draw_102,...,draw_991,draw_992,draw_993,draw_994,draw_995,draw_996,draw_997,draw_998,draw_999,measure
0,6,2021.0,wheat flour,0.2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,pif
1,6,2021.0,wheat flour,0.5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,pif
2,6,2021.0,wheat flour,0.8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,pif
3,6,2022.0,wheat flour,0.2,2.893671,1.469553,2.820851,1.531241,3.432355,3.164135,...,2.901130,3.510161,4.013109,1.772386,2.407369,3.790683,3.179377,1.482380,2.352735,pif
4,6,2022.0,wheat flour,0.5,7.234177,3.673881,7.052127,3.828102,8.580887,7.910337,...,7.252825,8.775402,10.032771,4.430964,6.018423,9.476707,7.948443,3.705950,5.881837,pif
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2875,522,2025.0,oil,0.5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,rates_averted
2876,522,2025.0,oil,0.8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,rates_averted
2877,522,2025.0,wheat flour,0.2,17.904824,19.916017,6.570495,16.546826,6.931337,7.607677,...,12.776907,51.948155,39.101503,21.942302,12.144394,11.844058,12.208273,10.141324,8.577025,rates_averted
2878,522,2025.0,wheat flour,0.5,44.762060,49.790042,16.426237,41.367065,17.328342,19.019193,...,31.942268,129.870386,97.753757,54.855754,30.360985,29.610144,30.520682,25.353311,21.442562,rates_averted


# ZINC

In [54]:
# ZINC
rei_id = 97
nutrient = 'zinc'
vehicles = ['wheat flour','maize flour']
effective_fractions = [0,0,0,1]

In [55]:
# define no fortification relative risk distribution
# vitamin a specific -- this should be replaced for other models

from numpy import log
from scipy.stats import norm, lognorm

# median and 0.975-quantile of lognormal distribution for RR
median = 0.47
q_975 = 0.69

# 0.975-quantile of standard normal distribution (=1.96, approximately)
q_975_stdnorm = norm().ppf(0.975)

mu = log(median) # mean of normal distribution for log(RR)
sigma = (log(q_975) - mu) / q_975_stdnorm # std dev of normal distribution for log(RR)

# Frozen lognormal distribution for RR, representing uncertainty in our effect size
# (s is the shape parameter)
rr_distribution = lognorm(s=sigma, scale=median)

In [56]:
zinc_baseline_coverage = pd.read_pickle('results_raw/baseline_coverage_data_u5.pkl')
zinc_baseline_coverage = (zinc_baseline_coverage.loc[zinc_baseline_coverage.nutrient==nutrient]
                              .loc[zinc_baseline_coverage.vehicle.isin(vehicles)]
                              .drop(columns='nutrient'))

zinc_counterfactual_coverage = pd.read_pickle('results_raw/counterfactual_coverage_data_u5.pkl')
zinc_counterfactual_coverage = (zinc_counterfactual_coverage.loc[zinc_counterfactual_coverage.nutrient==nutrient]
                              .loc[zinc_counterfactual_coverage.vehicle.isin(vehicles)]
                                    .drop(columns='nutrient'))

zinc_baseline_coverage.head()

,draw_0,draw_1,draw_10,draw_100,draw_101,draw_102,draw_103,draw_104,draw_105,draw_106,...,draw_993,draw_994,draw_995,draw_996,draw_997,draw_998,draw_999,location_id,year,vehicle
1344,0.048558,0.040897,0.057464,0.055794,0.078894,0.069322,0.066442,0.074779,0.040128,0.051428,...,0.060894,0.038659,0.063932,0.084612,0.040307,0.07357,0.062551,6,2021.0,wheat flour
1345,0.048558,0.040897,0.057464,0.055794,0.078894,0.069322,0.066442,0.074779,0.040128,0.051428,...,0.060894,0.038659,0.063932,0.084612,0.040307,0.07357,0.062551,6,2022.0,wheat flour
1346,0.048558,0.040897,0.057464,0.055794,0.078894,0.069322,0.066442,0.074779,0.040128,0.051428,...,0.060894,0.038659,0.063932,0.084612,0.040307,0.07357,0.062551,6,2023.0,wheat flour
1347,0.048558,0.040897,0.057464,0.055794,0.078894,0.069322,0.066442,0.074779,0.040128,0.051428,...,0.060894,0.038659,0.063932,0.084612,0.040307,0.07357,0.062551,6,2024.0,wheat flour
1348,0.048558,0.040897,0.057464,0.055794,0.078894,0.069322,0.066442,0.074779,0.040128,0.051428,...,0.060894,0.038659,0.063932,0.084612,0.040307,0.07357,0.062551,6,2025.0,wheat flour


In [57]:
zinc_rr_deficiency_nofort = (1 / generate_rr_deficiency_nofort_draws(mu, sigma, location_ids))
zinc_rr_deficiency_nofort.mean(axis=1)

location_id
179    2.178029
163    2.178029
214    2.178029
161    2.178029
165    2.178029
190    2.178029
189    2.178029
196    2.178029
201    2.178029
180    2.178029
15     2.178029
164    2.178029
20     2.178029
202    2.178029
205    2.178029
171    2.178029
11     2.178029
184    2.178029
168    2.178029
6      2.178029
141    2.178029
207    2.178029
181    2.178029
213    2.178029
522    2.178029
dtype: float64

In [58]:
zinc_dalys_hdf_key = '/zinc_deficiency/dalys_attributable_bmgf_25_countries'
try:
    zinc_dalys = pd.read_hdf(gbd_data_filepath, key=zinc_dalys_hdf_key)
except (FileNotFoundError, KeyError) as e:
    print(e, "Calling get_draws...")
    zinc_dalys = pull_deficiency_attributable_dalys(rei_id,
                                              location_ids,
                                              age_group_ids,
                                              sex_ids,
                                              index_cols)
else:
    # Stored data included all 25 locations, so filter to desired locations if try clause succeeded
    zinc_dalys = zinc_dalys.query("location_id in @location_ids")

zinc_dalys.head()

draw_0      draw_1     draw_10  \
location_id sex_id age_group_id                                        
6           1      5               55.806889    0.000000   29.211177   
            2      5               37.542436    0.000000   21.781172   
11          1      5             1057.924493  369.707946  517.346114   
            2      5              954.532812  197.702087  436.339271   
15          1      5              313.163965   18.266017   79.234558   

                                    draw_100    draw_101    draw_102  \
location_id sex_id age_group_id                                        
6           1      5               91.031178   45.761789   29.550910   
            2      5               86.086341   38.032330   24.744728   
11          1      5             1121.865646  383.960301  326.811523   
            2      5             1209.191167  371.705455  208.350528   
15          1      5              128.724069   27.397756   42.252433   

                                   draw_103     draw_104     draw_105  \
location_id sex_id age_group_id                                         
6           1      5              11.862744    39.110218   142.270449   
            2      5               8.679425    33.560425   124.455756   
11          1      5             290.876450  1234.791858  1980.744828   
            2      5             464.778852  1270.639991  1371.653872   
15          1      5              45.034578   168.170188   167.670791   

                                   draw_106  ...     draw_990     draw_991  \
location_id sex_id age_group_id              ...                             
6           1      5              27.921471  ...    70.827393   154.927527   
            2      5              18.714235  ...    63.459794   123.302919   
11          1      5             928.505912  ...  1295.128773  2676.259203   
            2      5             665.427560  ...  1074.683581  2096.088331   
15          1      5             189.359412  ...   132.672371   285.220362   

                                    draw_992     draw_993     draw_994  \
location_id sex_id age_group_id                                          
6           1      5               29.528407   146.462447   219.592170   
            2      5               20.750247   115.160657   196.325055   
11          1      5             1164.434543  1504.802755  3333.240053   
            2      5              897.944516   993.337161  2816.770108   
15          1      5              102.390704   220.484294   547.323432   

                                    draw_995     draw_996     draw_997  \
location_id sex_id age_group_id                                          
6           1      5              165.562467    62.928907    32.051667   
            2      5              154.511623    51.370814    35.211259   
11          1      5             2298.076536  1813.060392  1378.863465   
            2      5             2380.433080  1282.128241   957.580283   
15          1      5              387.181049   198.262244    86.842222   

                                    draw_998     draw_999  
location_id sex_id age_group_id                            
6           1      5               77.849291   149.179710  
            2      5               63.368359   129.279112  
11          1      5             1424.566740  1897.192422  
            2      5             1063.576027  1460.532078  
15          1      5              112.237688   146.724520  

[5 rows x 1000 columns]

In [59]:
zinc_baseline_effective_coverage = apply_age_related_effective_coverage_restrictions(
                                                                zinc_baseline_coverage.set_index([c for c in zinc_baseline_coverage.columns if 'draw' not in c]),
                                                                sex_ids,
                                                                age_group_ids,
                                                                effective_fractions)
zinc_counterfactual_effective_coverage = apply_age_related_effective_coverage_restrictions(
                                                                zinc_counterfactual_coverage.set_index([c for c in zinc_counterfactual_coverage.columns if 'draw' not in c]),
                                                                sex_ids,
                                                                age_group_ids,
                                                                effective_fractions)

In [60]:
zinc_paf_deficiency_nofort = calculate_paf_deficiency_nofort(
                                    zinc_rr_deficiency_nofort, 
                                    zinc_baseline_effective_coverage)
zinc_paf_deficiency_nofort.head()

draws                                                 draw_0    draw_1  \
location_id age_group_id sex_id year   vehicle                           
6           2            1      2021.0 wheat flour  0.345476  0.571001   
                                2022.0 wheat flour  0.345476  0.571001   
                                2023.0 wheat flour  0.345476  0.571001   
                                2024.0 wheat flour  0.345476  0.571001   
                                2025.0 wheat flour  0.345476  0.571001   

draws                                                draw_10  draw_100  \
location_id age_group_id sex_id year   vehicle                           
6           2            1      2021.0 wheat flour  0.584197  0.603221   
                                2022.0 wheat flour  0.584197  0.603221   
                                2023.0 wheat flour  0.584197  0.603221   
                                2024.0 wheat flour  0.584197  0.603221   
                                2025.0 wheat flour  0.584197  0.603221   

draws                                               draw_101  draw_102  \
location_id age_group_id sex_id year   vehicle                           
6           2            1      2021.0 wheat flour  0.494235   0.49253   
                                2022.0 wheat flour  0.494235   0.49253   
                                2023.0 wheat flour  0.494235   0.49253   
                                2024.0 wheat flour  0.494235   0.49253   
                                2025.0 wheat flour  0.494235   0.49253   

draws                                               draw_103  draw_104  \
location_id age_group_id sex_id year   vehicle                           
6           2            1      2021.0 wheat flour  0.645743  0.483019   
                                2022.0 wheat flour  0.645743  0.483019   
                                2023.0 wheat flour  0.645743  0.483019   
                                2024.0 wheat flour  0.645743  0.483019   
                                2025.0 wheat flour  0.645743  0.483019   

draws                                               draw_105  draw_106  ...  \
location_id age_group_id sex_id year   vehicle                          ...   
6           2            1      2021.0 wheat flour  0.579614  0.378442  ...   
                                2022.0 wheat flour  0.579614  0.378442  ...   
                                2023.0 wheat flour  0.579614  0.378442  ...   
                                2024.0 wheat flour  0.579614  0.378442  ...   
                                2025.0 wheat flour  0.579614  0.378442  ...   

draws                                               draw_990  draw_991  \
location_id age_group_id sex_id year   vehicle                           
6           2            1      2021.0 wheat flour  0.548122  0.568194   
                                2022.0 wheat flour  0.548122  0.568194   
                                2023.0 wheat flour  0.548122  0.568194   
                                2024.0 wheat flour  0.548122  0.568194   
                                2025.0 wheat flour  0.548122  0.568194   

draws                                               draw_992  draw_993  \
location_id age_group_id sex_id year   vehicle                           
6           2            1      2021.0 wheat flour  0.399642   0.34575   
                                2022.0 wheat flour  0.399642   0.34575   
                                2023.0 wheat flour  0.399642   0.34575   
                                2024.0 wheat flour  0.399642   0.34575   
                                2025.0 wheat flour  0.399642   0.34575   

draws                                               draw_994  draw_995  \
location_id age_group_id sex_id year   vehicle                           
6           2            1      2021.0 wheat flour  0.587321  0.526427   
                                2022.0 wheat flour  0.587321  0.526427   
                                2023

In [61]:
zinc_pif_deficiency_nofort = calculate_pif_deficiency_nofort(
                                    zinc_paf_deficiency_nofort,
                                    zinc_baseline_effective_coverage,
                                    zinc_counterfactual_effective_coverage)
zinc_pif_deficiency_nofort.head()

draws                                                              draw_0  \
location_id age_group_id sex_id year   vehicle     coverage_level           
6           2            1      2021.0 wheat flour 0.2                0.0   
                                                   0.5                0.0   
                                                   0.8                0.0   
                                2022.0 wheat flour 0.2                0.0   
                                                   0.5                0.0   

draws                                                              draw_1  \
location_id age_group_id sex_id year   vehicle     coverage_level           
6           2            1      2021.0 wheat flour 0.2                0.0   
                                                   0.5                0.0   
                                                   0.8                0.0   
                                2022.0 wheat flour 0.2                0.0   
                                                   0.5                0.0   

draws                                                              draw_10  \
location_id age_group_id sex_id year   vehicle     coverage_level            
6           2            1      2021.0 wheat flour 0.2                 0.0   
                                                   0.5                 0.0   
                                                   0.8                 0.0   
                                2022.0 wheat flour 0.2                 0.0   
                                                   0.5                 0.0   

draws                                                              draw_100  \
location_id age_group_id sex_id year   vehicle     coverage_level             
6           2            1      2021.0 wheat flour 0.2                  0.0   
                                                   0.5                  0.0   
                                                   0.8                  0.0   
                                2022.0 wheat flour 0.2                  0.0   
                                                   0.5                  0.0   

draws                                                              draw_101  \
location_id age_group_id sex_id year   vehicle     coverage_level             
6           2            1      2021.0 wheat flour 0.2                  0.0   
                                                   0.5                  0.0   
                                                   0.8                  0.0   
                                2022.0 wheat flour 0.2                  0.0   
                                                   0.5                  0.0   

draws                                                              draw_102  \
location_id age_group_id sex_id year   vehicle     coverage_level             
6           2            1      2021.0 wheat flour 0.2                  0.0   
                                                   0.5                  0.0   
                                                   0.8                  0.0   
                                2022.0 wheat flour 0.2                  0.0   
                                                   0.5                  0.0   

draws                                                              draw_103  \
location_id age_group_id sex_id year   vehicle     coverage_level             
6           2            1      2021.0 wheat flour 0.2                  0.0   
                                                   0.5                  0.0   
                                                   0.8                  0.0   
                                2022.0 wheat flour 0.2                  0.0   
                                                   0.5                  0.0   

draws                                                              draw_104  \
location_id age_group_id sex_id year   vehicle     coverage_level             

In [62]:
zinc_overall_pifs, zinc_daly_reduction_counts = calculate_final_pifs_and_daly_reductions(
                                                    zinc_pif_deficiency_nofort,
                                                    zinc_dalys,
                                                    coverage_levels, years)

In [63]:
zinc_daly_reduction_rates = calculate_rates(zinc_daly_reduction_counts, 
                                            location_ids, 
                                            age_group_ids, 
                                            sex_ids)

In [64]:
zinc_overall_pifs['measure'] = 'pif'
zinc_daly_reduction_counts['measure'] = 'counts_averted'
zinc_daly_reduction_rates['measure'] = 'rates_averted'
zinc_results = pd.concat([zinc_overall_pifs.reset_index(),
                         zinc_daly_reduction_counts.reset_index(),
                         zinc_daly_reduction_rates.reset_index()],
                        ignore_index=True)
zinc_results.to_pickle('results_raw/zinc.pkl')

In [65]:
zinc_exposure = get_draws(gbd_round_id=6,
                              year_id=2019,
                              location_id=location_ids,
                              sex_id=sex_ids,
                              age_group_id=age_group_ids,
                              gbd_id_type='rei_id',
                              gbd_id=97,
                              source='exposure',
                              decomp_step='step4')

zinc_exp = (zinc_exposure.loc[zinc_exposure.parameter=='cat1']
                 .set_index(['location_id','age_group_id','sex_id']))
zinc_exp = zinc_exp.drop(columns=[c for c in zinc_exp.columns if 'draw' not in c])

zinc_exp = zinc_exp - (zinc_exp * zinc_pif_deficiency_nofort)

zinc_exp = zinc_exp.reset_index().merge(get_population(gbd_round_id=6,
                    location_id=location_ids,
                    sex_id=sex_ids,
                    age_group_id=age_group_ids,
                    year_id=2019,
                    decomp_step='step4'), on=['location_id','sex_id','age_group_id'])

for i in list(range(0,1000)):
    zinc_exp[f'draw_{i}'] = zinc_exp[f'draw_{i}'] * zinc_exp.population
    
zinc_exp = zinc_exp.groupby(['location_id','vehicle','year','coverage_level']).sum()

for i in list(range(0,1000)):
    zinc_exp[f'draw_{i}'] = zinc_exp[f'draw_{i}'] / zinc_exp.population * 100
    
zinc_exp = (zinc_exp.drop(columns=[c for c in zinc_exp.columns if 'draw' not in c])
                 .reset_index())
zinc_exp = zinc_exp.loc[zinc_exp.year==2025]
zinc_exp['nutrient'] = 'zinc'

zinc_exp

,location_id,vehicle,year,coverage_level,draw_0,draw_1,draw_10,draw_100,draw_101,draw_102,...,draw_991,draw_992,draw_993,draw_994,draw_995,draw_996,draw_997,draw_998,draw_999,nutrient
12,6,wheat flour,2025.0,0.2,0.362598,0.000000,0.227301,0.566955,0.551045,0.442204,...,1.467234,0.109735,0.732990,1.213139,1.562912,0.530769,0.262585,0.656613,1.737130,zinc
13,6,wheat flour,2025.0,0.5,0.348445,0.000000,0.208738,0.520522,0.498003,0.405418,...,1.322629,0.101974,0.696680,1.148046,1.434921,0.477845,0.248241,0.586134,1.580213,zinc
14,6,wheat flour,2025.0,0.8,0.334291,0.000000,0.190174,0.474089,0.444961,0.368633,...,1.178024,0.094213,0.660370,1.082953,1.306929,0.424921,0.233898,0.515654,1.423296,zinc
27,15,wheat flour,2025.0,0.2,3.660934,0.388549,2.047912,2.854206,1.517485,1.120029,...,8.309265,1.977538,2.637141,8.592320,8.857993,4.175807,2.595378,4.226171,8.306975,zinc
28,15,wheat flour,2025.0,0.5,3.647227,0.386025,2.033426,2.834228,1.508316,1.114632,...,8.256512,1.970515,2.626785,8.531223,8.812050,4.151878,2.580534,4.202682,8.257392,zinc
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
598,522,maize flour,2025.0,0.5,26.152533,18.500809,23.043904,23.354591,21.937244,19.555100,...,29.969299,24.037032,23.869386,29.067841,33.006204,27.624516,23.498428,25.323412,29.913192,zinc
599,522,maize flour,2025.0,0.8,25.861884,17.488839,22.209073,22.058662,21.177138,18.911884,...,28.305910,23.556857,23.033106,27.603781,31.863956,25.877262,21.589295,24.133924,28.775398,zinc
612,522,wheat flour,2025.0,0.2,25.980008,18.906792,23.108654,23.624401,21.527439,19.412005,...,31.199522,24.121851,24.595403,28.653178,33.045138,28.656187,24.505657,25.224151,30.636845,zinc
613,522,wheat flour,2025.0,0.5,24.994598,16.985842,21.118699,20.789293,19.012467,17.589322,...,28.886383,23.048640,23.593728,24.371033,30.247918,25.835556,21.243670,22.101538,28.877838,zinc


In [66]:
zinc_exp.to_pickle('results_raw/zinc_deficiency_exposure.pkl')